# Challenge: Múltiples variables almacenadas en una columna.

Vamos a tratar ahora con el siguiente dataset: Registros de tuberculosis de la Organización Mundial de la Salud
Este conjunto de datos documenta el recuento de casos confirmados de tuberculosis por país, año, edad y sexo.

## Problemas:
- Algunas columnas contienen varios valores: sexo y edad.
- Mezcla de ceros y valores perdidos NaN. Esto se debe al proceso de recopilación de datos y la distinción es importante para este conjunto de datos.

In [2]:
import pandas as pd

In [3]:
tb = pd.read_csv("tb-raw.csv")
tb

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,NaN,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,NaN,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,NaN,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,NaN,3.0
5,AM,2000,2.0,152.0,130.0,131.0,63,26,21.0,NaN,1.0
6,AN,2000,0.0,0.0,1.0,2.0,0,0,0.0,NaN,0.0
7,AO,2000,186.0,999.0,1003.0,912.0,482,312,194.0,NaN,247.0
8,AR,2000,97.0,278.0,594.0,402.0,419,368,330.0,NaN,121.0
9,AS,2000,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN


In [4]:
tb.drop(columns=["mu"], inplace=True) #Eliminamos columna de datos nulos
tb

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,3.0
5,AM,2000,2.0,152.0,130.0,131.0,63,26,21.0,1.0
6,AN,2000,0.0,0.0,1.0,2.0,0,0,0.0,0.0
7,AO,2000,186.0,999.0,1003.0,912.0,482,312,194.0,247.0
8,AR,2000,97.0,278.0,594.0,402.0,419,368,330.0,121.0
9,AS,2000,NaN,NaN,NaN,NaN,1,1,NaN,NaN


Para ordenar este conjunto de datos, necesitamos eliminar los diferentes valores del encabezado y SEPARARLOS en filas.

### Instrucciones:
- Primero, necesitaremos fusionar las columnas de sexo + grupo de edad en una sola.


In [5]:
#Se utilizó la función melt
tb1=pd.melt(frame=tb, id_vars=["country","year"], var_name="sex + grupo de edad", value_name="cases")
tb1

,country,year,sex + grupo de edad,cases
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0
...,...,...,...,...
75,AM,2000,f014,1.0
76,AN,2000,f014,0.0
77,AO,2000,f014,247.0
78,AR,2000,f014,121.0


- Segundo, separamos el string de los integers de la columa "sexo + grupo de edad", lo que nos dará como resultado dos columnas "sexo" y "grupo de edad"

In [6]:
#Se utilizó la función drop y split
tb1Expanded=tb1["sex + grupo de edad"].str.split("(\d+)", expand=True)
tb1Expanded.drop(columns=[2],inplace=True) #Se elimino la columna 2 porque no tenía ninguna observación.
tb1Expanded.columns=["sex","grupo de edad"]
tb1Expanded

,sex,grupo de edad
0,m,014
1,m,014
2,m,014
3,m,014
4,m,014
...,...,...
75,f,014
76,f,014
77,f,014
78,f,014


- Tercero, eliminamos la columna de "sexo + grupo de edad" de la tabla original y las sustituimos por las columnas credas en el punto anterior.

In [7]:
#Función concat
tb1=tb1.drop(columns=["sex + grupo de edad"])
tb2=pd.concat([tb1, tb1Expanded ],axis=1)
tb2

,country,year,cases,sex,grupo de edad
0,AD,2000,0.0,m,014
1,AE,2000,2.0,m,014
2,AF,2000,52.0,m,014
3,AG,2000,0.0,m,014
4,AL,2000,2.0,m,014
...,...,...,...,...,...
75,AM,2000,1.0,f,014
76,AN,2000,0.0,f,014
77,AO,2000,247.0,f,014
78,AR,2000,121.0,f,014


- Cuarto, necesitamos definir los rangos de edad separando el -Age lower- del -Age upper- por un guión "-".
Para ello primero separamos los valores de la columna "grupo de edad" con la función "split".

In [8]:
tb2_age=tb2["grupo de edad"].str.split("", expand=True)
tb2_age.drop(columns=[0,5],inplace=True)   #Eliminamos los columnas 0 y 5 porque no contienen datos.
tb2_age["age"]=0 #Agregamos una columna extra "Age", la cual nos servirá para definir los rangos de edad.
tb2_age

,1,2,3,4,age
0,0,1,4,,0
1,0,1,4,,0
2,0,1,4,,0
3,0,1,4,,0
4,0,1,4,,0
...,...,...,...,...,...
75,0,1,4,,0
76,0,1,4,,0
77,0,1,4,,0
78,0,1,4,,0


In [9]:
# Para definir las funciones se utilizó un for combinado con un if para 
for i in range(len(tb2_age)):
    if  tb2["grupo de edad"][i]=='65':
        tb2_age["age"][i]="65"
    elif tb2["grupo de edad"][i]=='014':
        tb2_age["age"][i]="0-14"
    else:
        tb2_age["age"][i]=(tb2_age[1][i]+tb2_age[2][i]+"-"+tb2_age[3][i]+tb2_age[4][i])
       

C:\Users\sabri\AppData\Local\Temp\ipykernel_12116\3925001918.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tb2_age["age"][i]="0-14"


In [10]:
tb2_age["age"].values #Comprobación de la definición de los rangos

array(['0-14', '0-14', '0-14', '0-14', '0-14', '0-14', '0-14', '0-14',
       '0-14', '0-14', '15-24', '15-24', '15-24', '15-24', '15-24',
       '15-24', '15-24', '15-24', '15-24', '15-24', '25-34', '25-34',
       '25-34', '25-34', '25-34', '25-34', '25-34', '25-34', '25-34',
       '25-34', '35-44', '35-44', '35-44', '35-44', '35-44', '35-44',
       '35-44', '35-44', '35-44', '35-44', '45-54', '45-54', '45-54',
       '45-54', '45-54', '45-54', '45-54', '45-54', '45-54', '45-54',
       '55-64', '55-64', '55-64', '55-64', '55-64', '55-64', '55-64',
       '55-64', '55-64', '55-64', '65', '65', '65', '65', '65', '65',
       '65', '65', '65', '65', '0-14', '0-14', '0-14', '0-14', '0-14',
       '0-14', '0-14', '0-14', '0-14', '0-14'], dtype=object)

-Quinto, unimos la nueva columna "age" a nuestra anterior tabla, y la reordenamos por los criterios de "country", "sex" y "age"

In [11]:
tb_f=pd.concat([tb2, tb2_age["age"]],axis=1) #Unimos la nueva columa
tb_f.drop(columns=["grupo de edad"],inplace=True) #Elimanos la anterior columna para los rangos de edad
tb_f1=tb_f[tb_f["cases"].notna()]  #Eliminamos datos NaN
tb_f1

,country,year,cases,sex,age
0,AD,2000,0.0,m,0-14
1,AE,2000,2.0,m,0-14
2,AF,2000,52.0,m,0-14
3,AG,2000,0.0,m,0-14
4,AL,2000,2.0,m,0-14
...,...,...,...,...,...
74,AL,2000,3.0,f,0-14
75,AM,2000,1.0,f,0-14
76,AN,2000,0.0,f,0-14
77,AO,2000,247.0,f,0-14


In [19]:
tb_f1['cases'] = tb_f1['cases'].astype(int) #Conversión de datos a int
tb_f1.sort_values(["country","sex","age"],ascending=True) #Tabla final

C:\Users\sabri\AppData\Local\Temp\ipykernel_12116\4083444278.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tb_f1['cases'] = tb_f1['cases'].astype(int) #Conversión de datos a int


,country,year,cases,sex,age
0,AD,2000,0,m,0-14
10,AD,2000,0,m,15-24
20,AD,2000,1,m,25-34
30,AD,2000,0,m,35-44
40,AD,2000,0,m,45-54
...,...,...,...,...,...
48,AR,2000,419,m,45-54
58,AR,2000,368,m,55-64
68,AR,2000,330,m,65
49,AS,2000,1,m,45-54


## Resultado
Al finalizar, deberías obtener un dataframe similar a este:

![image.png](https://storage.googleapis.com/campus-cvs/lectures/tidyDataChallenge2.PNG)

Listo!